In [ ]:
import cv2
import mediapipe as mp
import pickle
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [3]:
model_dict = pickle.load(open('../models/model.pickle', 'rb'))
data_dict = pickle.load(open('../models/data.pickle', 'rb'))
model = model_dict['model']

In [4]:
labels = np.unique(data_dict['labels'])
print("Available classes:", labels)

Available classes: ['Hello' 'IloveYou' 'No' 'Please' 'Thanks' 'Yes']


In [5]:
max_len = max(len(l) for l in data_dict['data'])

In [6]:
mp_hands = mp.solutions.hands  # type: ignore[attr-defined]
mp_drawing = mp.solutions.drawing_utils  # type: ignore[attr-defined]
mp_drawing_styles = mp.solutions.drawing_styles  # type: ignore[attr-defined]

hands = mp_hands.Hands(static_image_mode=False,
                       max_num_hands=2,
                       min_detection_confidence=0.3)

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        print("❌ Failed to grab frame. Check camera index.")
        break

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)

    if results.multi_hand_landmarks:
        data_aux = []

        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                frame,
                hand_landmarks,
                mp_hands.HAND_CONNECTIONS,
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style()
            )
            for lm in hand_landmarks.landmark:
                data_aux.extend([lm.x, lm.y])

        if len(data_aux) < max_len:
            data_aux.extend([0] * (max_len - len(data_aux)))
        elif len(data_aux) > max_len:
            data_aux = data_aux[:max_len]

        prediction = model.predict([np.asarray(data_aux)])
        predicted_sign = prediction[0]

        cv2.putText(frame, f"Prediction: {predicted_sign}", (10, 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow('Hand Sign Detection', frame)

    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()